Bloque 1: Bibliotecas

In [ ]:
import importlib

import RedFunciones.visualizacion as visualizacion
import RedFunciones.Generador as Generador
import RedFunciones.Discriminador as Discriminador
import RedFunciones.auxiliares as auxiliares
import RedFunciones.DataloaderVol as DataloaderVol
import RedFunciones.Checkpoint as Checkpoint

importlib.reload(auxiliares)
importlib.reload(visualizacion)
importlib.reload(Generador)
importlib.reload(Discriminador)
importlib.reload(DataloaderVol)
importlib.reload(Checkpoint)

import Auxiliares.BinaryAccuracy as bin_acc

import json

import PrePross.pre_pross as prpr
import PrePross.grifflin as grifflin
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import importlib
import pandas as pd
importlib.reload(prpr)

import torch
from torch import nn
from tqdm.auto import tqdm 

from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.datasets import MNIST 

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None 

import os
cwd = os.getcwd()
print(cwd)

cuda0 = torch.device('cuda:0')
cpu = 'cpu'
device = torch.device(cuda0 if torch.cuda.is_available() else cpu)
print(torch.cuda.is_available())
print(torch.cuda.device_count())


Bloque 2:Pre Pros
generar_nueva_base: Realizar el proceso de pre procesamiento y generar una nueva base pre procesada 

limite_snr: Limite de filtro de SNR en el pre procesamiento 

In [ ]:
generar_nueva_base = False
if (generar_nueva_base):
    limite_snr = 10
    base_vt=prpr.open_bdd('Data/VT_IGEPN_1.json')
    base_lp=prpr.open_bdd('Data/LP_IGEPN_1.json')
    basef = pd.concat([base_vt, base_lp])
    base=prpr.extraer_señales(basef, '')
    prpr.normalizar_tamanio_base(base, 83)
    prpr.normalizar_muestras(base, 8192/2)
    base_rms=prpr.quitar_dc(basef)
    base = base_rms
    base = prpr.drop_data_na(base)
    base = prpr.drop_data_ruido(base, limite_snr)
    base = prpr.normailizar_muestras(base)
    prpr.guardar_base_h5(base,129,33, nombre='baseSR10')

Bloque 3: Semilla para resultados determinísticos  

In [ ]:
seed = 0
torch.manual_seed(seed) 
torch.cuda.manual_seed_all(seed)  
torch.backends.cudnn.deterministic = True  
torch.backends.cudnn.benchmark = True

In [ ]:
display_step = 250

h5_filename = cwd + '/baseh5/baseSR10.h5'
nombre = 'master'
nombre_carga = 'final15.pt'

z_dim = 64
n_epochs = 15
batch_size = 12

lrg = 0.00009
beta_1_g = 0.5
beta_2_g = 0.999
repeats_g = 3

lrd = 0.0002
beta_1_d = 0.5
beta_2_d = 0.999
d_repeats = 1

guardar = False
cargar = False
save_steep = 1

regularizaM = True
reg_lambda = 0.001

Bloque 5: Constantes 

In [ ]:
criterion = nn.BCEWithLogitsLoss()
mnist_shape = (1, 129, 33) 
n_classes = 2
mean = 0
std = 1
epoch_temp = -1
gen_loss_temp = torch.empty([])
disc_loss_temp = torch.empty([])

Bloque 6: Dataset 

In [ ]:
transform = transforms.Compose([])    
myData = DataloaderVol.VolcanoDatasetH5(h5_filename, transform)

dataloader = DataLoader(
    myData,
    batch_size=batch_size,
    shuffle=True)

Bloque 7: Declaración de elementos de cGAN 

In [ ]:
generator_input_dim, discriminator_im_chan = auxiliares.get_input_dimensions(z_dim, mnist_shape, n_classes)

gen = Generador.Generator(input_dim=generator_input_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lrg)
disc = Discriminador.Discriminator(im_chan=discriminator_im_chan).to(device)
disc_opt = torch.optim.Adam(disc.parameters(), lr=lrd)
gen = gen.apply(auxiliares.weights_init)
disc = disc.apply(auxiliares.weights_init)

disc.train()
gen.train()
        
metric = bin_acc.BinaryAccuracy()

Bloque 8: Carga checkpoint 

In [ ]:
if cargar:
    compelto = Checkpoint.DIRECTORY + nombre_carga
    checkpoint = torch.load(compelto)
    gen.load_state_dict(checkpoint['gen'])
    disc.load_state_dict(checkpoint['disc'])
    gen_opt.load_state_dict(checkpoint['gen_opt'])
    disc_opt.load_state_dict(checkpoint['disc_opt'])
    epoch_temp = checkpoint['epoch']
    gen_loss_temp = checkpoint['gen_loss']
    disc_loss_temp = checkpoint['dis_loss']
    print('Modelo cargado')
    print('epoch: ' + str(epoch_temp))

Bloque 9: Entrenamiento  

In [ ]:
cur_step = 0
generator_losses = []
discriminator_losses = []
discriminator_losses_real = []
discriminator_losses_fake = []
metric_real = []
metric_fake = [] 

for epoch in range(epoch_temp + 1, n_epochs):
    print('Epoch: ' + str(epoch))
    for real, labels in tqdm(dataloader):
        cur_batch_size = len(real)
        real = real.to(device)
        one_hot_labels = auxiliares.get_one_hot_labels(labels.to(device), n_classes)
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = image_one_hot_labels.repeat(1, 1, mnist_shape[1], mnist_shape[2])
        disc_opt.zero_grad()
        fake_noise = auxiliares.get_noise(cur_batch_size, z_dim, device=device)
        noise_and_labels = auxiliares.combine_vectors(fake_noise, one_hot_labels)
        fake = gen.forward(noise_and_labels)    
        #fake = gen(noise_and_labels)#gen.forward(noise_and_labels)
        fake_image_and_labels = auxiliares.combine_vectors(fake.detach(), image_one_hot_labels)
        real_image_and_labels = auxiliares.combine_vectors(real, image_one_hot_labels)
        disc_fake_pred = disc(fake_image_and_labels)
        disc_real_pred = disc(real_image_and_labels)               
        disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
        disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
        metric_fake += [metric(disc_fake_pred, torch.zeros_like(disc_fake_pred)).item()]
        metric_real += [metric(disc_real_pred, torch.ones_like(disc_real_pred)).item()]       
        disc_loss = ((disc_fake_loss + disc_real_loss)/2)       
        if regularizaM:
            l2_reg = None
            for W in disc.parameters():
                if l2_reg is None:
                    l2_reg = W.norm(2)
                else:
                    l2_reg = l2_reg + W.norm(2)
            disc_loss = disc_loss +  l2_reg * reg_lambda         
        disc_loss.backward(retain_graph=True)
        disc_opt.step() 
        discriminator_losses_real += [disc_real_loss.item()]
        discriminator_losses_fake+= [disc_fake_loss.item()]
        discriminator_losses += [disc_loss.item()]
        mean_iteration_gen_loss = 0
        for _ in range(repeats_g):
            gen_opt.zero_grad()
            fake_noise_2 = auxiliares.get_noise(cur_batch_size, z_dim, device=device)       
            noise_and_labels_2 = auxiliares.combine_vectors(fake_noise_2, one_hot_labels)
            fake_2 = gen(noise_and_labels_2)   
            fake_image_and_labels = auxiliares.combine_vectors(fake_2, image_one_hot_labels)
            disc_fake_pred = disc(fake_image_and_labels)
            gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
            mean_iteration_gen_loss += gen_loss.item() / repeats_g    
            gen_loss.backward()
            gen_opt.step()
        generator_losses += [mean_iteration_gen_loss]
        if cur_step % display_step == 0 and cur_step > 0:
            gen_mean = sum(generator_losses[-display_step:]) / display_step
            disc_mean = sum(discriminator_losses[-display_step:]) / display_step
            print(f"Step {cur_step}: Generator loss: {gen_mean}, discriminator loss: {disc_mean}")
            print(f"Espectrogramas Generados")
            visualizacion.show_tensor_images(torch.transpose(fake, 2, 3), size=(1, 129, 33), num_images=1)
            print(f"Espectrogramas Reales")
            visualizacion.show_tensor_images(torch.transpose(real, 2, 3), size=(1, 129, 33), num_images=1)
            
            visualizacion.show_time_domine_images(torch.transpose(real, 2, 3), size=(1, 129, 33), std=std, mean=mean, real = True)
            visualizacion.show_time_domine_images(torch.transpose(fake, 2, 3), size=(1, 129, 33), std=std, mean=mean, real = False)
            step_bins = 20
            x_axis = sorted([i * step_bins for i in range(len(generator_losses) // step_bins)] * step_bins)
            num_examples = (len(generator_losses) // step_bins) * step_bins
            plt.plot(
                range(num_examples // step_bins), 
                torch.Tensor(generator_losses[:num_examples]).view(-1, step_bins).mean(1),
                label="Generator Loss"
            )
            plt.plot(
                range(num_examples // step_bins), 
                torch.Tensor(discriminator_losses[:num_examples]).view(-1, step_bins).mean(1),
                label="Discriminator Loss"
            )
            plt.legend()
            
            plt.show()
            plt.plot(
                range(num_examples // step_bins), 
                torch.Tensor(generator_losses[:num_examples]).view(-1, step_bins).mean(1),
                label="Generator Loss"
            )
            plt.plot(
                range(num_examples // step_bins), 
                torch.Tensor(discriminator_losses_fake[:num_examples]).view(-1, step_bins).mean(1),
                label="Discriminator Loss Fake"
            )
            plt.plot(
                range(num_examples // step_bins), 
                torch.Tensor(discriminator_losses_real[:num_examples]).view(-1, step_bins).mean(1),
                label="Discriminator Loss Real"
            )
            plt.legend()
            
            plt.show()
            plt.plot(
                range(num_examples // step_bins), 
                torch.Tensor(metric_real[:num_examples]).view(-1, step_bins).mean(1),
                label="METRIC REAL"
            )
            plt.plot(
                range(num_examples // step_bins), 
                torch.Tensor(metric_fake[:num_examples]).view(-1, step_bins).mean(1),
                label="METRIC FAKE"
            )
            plt.legend()
            plt.show()
        elif cur_step == 0:
            print("Red Funcionando")
        cur_step += 1 
    if((guardar and epoch % save_steep == 0) or epoch == n_epochs - 1):
        Checkpoint.save_weighs(gen, disc, gen_opt, disc_opt, epoch, gen_loss, disc_loss, nombre)
        print('epoch guardada')
gen.eval()
disc.eval()

Bloque 10: Generación de ejemplos  

In [ ]:
import PrePross.grifflin as grifflin
import numpy as np

examples = 3

for i in range(examples):
    x = torch.tensor([1])
    one_hot_labels = auxiliares.get_one_hot_labels(x.to(device), n_classes)
    fake_noise = auxiliares.get_noise(1, z_dim, device=device)
    noise_and_labels = auxiliares.combine_vectors(fake_noise, one_hot_labels)
    fake = gen(noise_and_labels)
    fake = fake.cpu().detach().numpy() 
    fake = fake * std + mean
    samplerate = 50
    timee, muestra_rec=grifflin.reconstruir_señal_generador(fake, 1000, samplerate)
    muestra_rec = np.squeeze(muestra_rec)
    tamaño = len(muestra_rec) / samplerate
    time = np.linspace(0., tamaño, len(muestra_rec))
    plt.plot(time,muestra_rec)
    plt.title("Señal Recuperada")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.show()
    x = torch.tensor([0])
    one_hot_labels = auxiliares.get_one_hot_labels(x.to(device), n_classes)
    fake_noise = auxiliares.get_noise(1, z_dim, device=device)
    noise_and_labels = auxiliares.combine_vectors(fake_noise, one_hot_labels)
    fake = gen(noise_and_labels)
    fake = fake.cpu().detach().numpy() 
    fake = fake * std + mean
    samplerate = 50
    timee, muestra_rec=grifflin.reconstruir_señal_generador(fake, 1000, samplerate)
    muestra_rec = np.squeeze(muestra_rec)
    tamaño = len(muestra_rec) / samplerate
    time = np.linspace(0., tamaño, len(muestra_rec))
    plt.plot(time,muestra_rec)
    plt.title("Señal Recuperada")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.show()
    

Bloque 11: Generación de muestras para calcular FID y calculo de la métrica FID 

In [ ]:
from torchvision.utils import save_image

n = 4792
for i in range(n):
    x = torch.tensor([1])
    one_hot_labels = auxiliares.get_one_hot_labels(x.to(device), n_classes)
    fake_noise = auxiliares.get_noise(1, z_dim, device=device)
    noise_and_labels = auxiliares.combine_vectors(fake_noise, one_hot_labels)
    fake = gen(noise_and_labels)
    nombre = 'img/fake/lp/img'+str(i)+'.png'
    save_image(fake, nombre)

n = 1358
for i in range(n):
    x = torch.tensor([0])
    one_hot_labels = auxiliares.get_one_hot_labels(x.to(device), n_classes)
    fake_noise = auxiliares.get_noise(1, z_dim, device=device)
    noise_and_labels = auxiliares.combine_vectors(fake_noise, one_hot_labels)
    fake = gen(noise_and_labels)
    nombre = 'img/fake/vt/img'+str(i)+'.png'
    save_image(fake, nombre)

n = 1000
for i in range(n):
    ruidoplano = np.random.normal(size = 129*33)
    ruido = ruidoplano.reshape((129, 33))
    nombre = 'img/ruido/img'+str(i)+'.png'
    sample = torch.from_numpy(ruido)
    save_image(sample, nombre)

n = myData.__len__()
c =0
cd=0
for i in range(n):
    sample, tag = myData.__getitem__(i)
    nombre = ''
    if tag == 0:
        nombre = 'img/real/vt/img'+str(i)+'.png'
        save_image(sample, nombre)
    if tag == 1:
        nombre = 'img/real/lp/img'+str(i)+'.png'
        save_image(sample, nombre)
!python -m pytorch_fid img/fake/vt img/real/vt
!python -m pytorch_fid img/fake/lp img/real/lp
!python -m pytorch_fid img/fake/vt img/ruido
!python -m pytorch_fid img/fake/lp img/ruido
!python -m pytorch_fid img/fake/vt img/fake/lp

Bloque 12: Generar base de datos de n muestras de cada tipo
numOfSamples: número de muestras de cada tipo a generar

In [ ]:
import h5py

fileName = 'baseh5/' + 'base_final' + '.h5'
numOfSamples = 10000
with h5py.File(fileName, "w") as out:
    dt = h5py.special_dtype(vlen=str)
    out.create_dataset("X_esp", (2*numOfSamples, 129, 33), dtype='float64')
    out.create_dataset("X_time", (2*numOfSamples, 4096, 1), dtype='float64')
    out.create_dataset("X_tag", (2*numOfSamples, 1), dtype=dt)
with h5py.File(fileName, "a") as out:
    for i in range(numOfSamples):
        x = torch.tensor([0])
        one_hot_labels = auxiliares.get_one_hot_labels(x.to(device), n_classes)
        fake_noise = auxiliares.get_noise(1, z_dim, device=device)
        noise_and_labels = auxiliares.combine_vectors(fake_noise, one_hot_labels)
        fake = gen(noise_and_labels)
        fake = fake.cpu().detach().numpy() 
        fake = fake * std + mean
        samplerate = 50
        timee, muestra_rec=grifflin.reconstruir_señal_generador(fake, 1000, samplerate)
        out['X_esp'][i, ...] = fake
        out['X_time'][i, ...] = np.squeeze(muestra_rec).reshape(4096, 1)
        out['X_tag'][i, ...] = 'VT'        
        x = torch.tensor([1])
        one_hot_labels = auxiliares.get_one_hot_labels(x.to(device), n_classes)
        fake_noise = auxiliares.get_noise(1, z_dim, device=device)
        noise_and_labels = auxiliares.combine_vectors(fake_noise, one_hot_labels)
        fake = gen(noise_and_labels)
        fake = fake.cpu().detach().numpy() 
        fake = fake * std + mean
        timee, muestra_rec=grifflin.reconstruir_señal_generador(fake, 1000, samplerate)
        out['X_esp'][i + 10000, ...] = fake
        out['X_time'][i + 10000, ...] = np.squeeze(muestra_rec).reshape(4096, 1)
        out['X_tag'][i + 10000, ...] = 'LP'
        if(i%10 == 0):
            print(i)